In [75]:
import cvxpy as cp
from datetime import datetime, timedelta

# Data manipulation
import numpy as np
import pandas as pd

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# Custom utilities and wrappers
import data_utils as du
import plotters

<a id="0"></a>

## <span style="color: #F8766D;">**Problem Formulation**</span>

So we have predicted the price in our previous question. Now, we want to make use of the predictions in order to maximize profit by charging and discharging our battery. We are given the following environment variables:

 - DAA Price (EUR/MWh) predicted by the XGBoost
 - A battery that can charge and discharge 1 MW per hour and has capacity of 1MWh 
 - <span style="color:rgb(20, 95, 12);">**Aim**</span>: to make the strategy when to charge and discharge in time span of last 24H available in dataset (i.e. 30. June 2022)

Furthermore, we will make the following <span style="color: #F8766D;">**assumptions:**</span>:

1. battery starts at capacity of 0
2. battery charge and discharge is instantaneous and incurs no transaction cost
3. battery does not degrade overtime
4. Energy market is open and always have supply for the battery to charge during the 24H
5. After the 24H, no more market. Battery must discharge before that.
6. At the beginning of the day, the strategy of charge/discharge is set for the 24H

**Remark**: one can think about this as buying a stock in the stock market where one person can buy or sell a homogenous stock given a period of time. The act of charging, would be buying a stock from the market and vice versa.

First, we define the denote the objective function as $f(x)$ by
    $$
    f(a) := \sum_{t}^T p(t) [- a(t)],
    $$
where $p(t)$ is the price at time $t$ and $a(t) \in [-1, 1, 0]$ is the action to charge/discharge at hourly time $t \in [1,\dots,T]$ (here $T=24$). In particular, the action space is given by
$$
a(t) =
\begin{cases}
1,\quad & \text{charge},\\
0, & \text{do nothing},\\
-1, & \text{discharge},
\end{cases}
$$
for all $t \in (1\dots,T)$

**Note** that in the objective function, the negative sign on the $a(t)$ is because as one discharges, $a(t) = -1$ and one earns profit, therefore it has to be negated.

Recall our aim is
    $$
    \max_{\phi(a)} f(a),
    $$
where $\phi(a) = (a(t))_{t=1}^T$ is schedule of charge/discharge.

Now we will formulate our constraints as follows:

1. The battery cannot be charged or discharged beyond its maximum rate, here 1 MHh:

   $$
   |a(t)| \leq 1.
   $$

2. Denote $s(t)$ to be the status of the battery at time $t$, its capacity can be represented by

   $$
   0 \leq s(t) \leq 1.
   $$

3. Recall we assumed we start with empty battery

    $$
    s(0) = 0.
    $$

4. To represent the physical fact that if we have an empty battery, we cannot discharge and that we must charged sometime earlier in order to discharge it later in time:
   
    $$
    \begin{align*}
    s(t) & = s(t-1) + a(t-1)
    \\
    & = s(t-2) + a(t-2) + a(t-1)
    \\
    & \vdots
    \\
    & = s(0) + \sum_{i=0}^{t-1} a(i)
    \end{align*}
    $$

(A bit of math) Finally, we need to prove that solution exists. Since our objective function is linear and, most importantly, that all the variable parameters above are closed and bounded (i.e. compact). Then, by Weierstrass Theorem, there must exist a unique maximum and minimum. Hence, the solution to the optimization problem exists and unique, and therefore well-defined.

## <span style="color: #F8766D;">**Code using CVXPY**</span>


In [76]:
# load the forecasted price by XGBoost in Q1
forecasted_df = pd.read_csv("data/xgb_forecasted_price_0.csv",
                            index_col='datetime'
                            )

plotters.plotly_actual_predict(
                        forecasted_df,
                        'price',
                        'price_prediction',
                        title='Day-ahead Price [EUR/MWh] Predicted with XGBoost'
                        )

In [77]:
# limits the scope to last 24H
df_24H = forecasted_df['price_prediction'][-24:]
predicted_array = df_24H.to_numpy() 

In [78]:
# Env parameters
def opt_battery_problem(
        predicted_array: np.ndarray,
        battery_capa: int = 1,
        max_charge_discharge: int = 1
        ) -> tuple[cp.Problem, np.ndarray, np.ndarray]:
    
    n = len(predicted_array)

    # define the optimization variables
    charge_discharge = cp.Variable(n)   # a(t)
    state_of_charge = cp.Variable(n)    # s(t)

    # constraints
    constraints = [
        state_of_charge[0] == 0,                    # Initial state of charge
        state_of_charge <= battery_capa,            # Battery capacity limit
        charge_discharge >= -max_charge_discharge,  # Maximum discharge rate
        charge_discharge <= max_charge_discharge,   # Maximum charge rate
        state_of_charge >= 0,                       # Battery cannot be negative
        state_of_charge[1:] == cp.cumsum(charge_discharge[:-1]),  # State of charge evolution
    ]

    # objective function
    objective = cp.Maximize(cp.sum(cp.multiply(predicted_array, -charge_discharge)))
    return cp.Problem(objective, constraints), charge_discharge, state_of_charge

In [83]:
# solve the problem
problem_obj_24H, charge_discharge, state_of_charge = opt_battery_problem(predicted_array)
problem_obj_24H.solve()

print(f"Maximum predicted profit: {round(problem_obj_24H.value,2)}")
print(f"Optimized charge/discharge schedule, phi(a): {np.round(charge_discharge.value)}")
print(f"State of charge at each time step, s(t): {np.round(state_of_charge.value)}")

Maximum predicted profit: 501.79
Optimized charge/discharge schedule, phi(a): [ 1. -0.  0.  0. -1.  1.  0. -0. -0. -1. -0.  0.  1. -0.  0. -0.  0. -0.
 -0. -1.  0. -0. -0. -1.]
State of charge at each time step, s(t): [ 0.  1.  1.  1.  1.  0.  1.  1.  1.  1.  0.  0.  0.  1.  1.  1.  1.  1.
  1.  1.  0.  0.  0. -0.]


In [84]:
px.line(x = df_24H.index,
        y=charge_discharge.value,
        title="Optimizeds schedule for charge/discharge, phi(a(t))")

In [85]:
px.line(x = df_24H.index,
        y=state_of_charge.value,
        title="Battery State, s(t)")

<span style="color:rgb(76, 45, 214);">**Note**</span>: Here we have obtained just the predicted profit according to the predicted pirce XGBoost modeled. To find the actual profit, we will need to compare with the real price with the strategy $\phi(a(t))$ we have optimized.

In [86]:
# load the real data
df_24H_actual = forecasted_df['price'][-24:]
df_24H_actual = df_24H_actual.to_numpy()

actual_profit = np.sum(- df_24H_actual * charge_discharge.value)
print(f"Actual profit: {round(actual_profit,2)}")

Actual profit: 507.24


#### <span style="color: #F8766D;">**Perfect Foresight**</span>

Let's first evaluate how much profit we would make if we knew the price in perfect foresight.


In [87]:
# solve the problem
problem_obj_act, charge_discharge_act, state_of_charge_act = opt_battery_problem(df_24H_actual)
problem_obj_act.solve()

print(f"Maximum profit: {round(problem_obj_act.value,2)}")
print(f"Optimized charge/discharge schedule, phi(a): {np.round(charge_discharge_act.value)}")
print(f"State of charge at each time step, s(t): {np.round(state_of_charge_act.value)}")


Maximum profit: 587.83
Optimized charge/discharge schedule, phi(a): [ 0.  1. -0. -1.  1.  0. -0. -0. -1. -0.  0.  1. -1.  1.  0.  0.  0. -0.
  0. -1. -0.  0. -0. -1.]
State of charge at each time step, s(t): [ 0.  0.  1.  1.  0.  1.  1.  1.  1.  0.  0.  0.  1.  0.  1.  1.  1.  1.
  1.  1.  0.  0.  0. -0.]


#### <span style="color: #F8766D;">**How did XGBoost + CVXPY do?**</span>

As shown above, we were able to get profit of 507,24 Euro with predicted price from XGBoost and optimization from CVXPY. When we have perfect foresight, we would have earn about 13,7% more.

### <span style="color: #F8766D;">**Limitations and Improvements**</span>



# Convexity Proof for Battery Optimization Problem

In the battery optimization problem, we aim to maximize profit by deciding on the charging and discharging strategy for a battery. The objective function and constraints must meet certain criteria to establish the problem as convex.

## Objective Function

The objective function is given by:

$$
\text{Maximize} \sum_{i=1}^{n} price[i] \cdot (-charge\_discharge[i])
$$

To prove convexity for a maximization problem, we need to show that the objective function is concave. However, we can minimize the negation of the profit function, leading to:

$$
\text{Minimize} \sum_{i=1}^{n} -price[i] \cdot charge\_discharge[i]
$$

Since the prices are constants and the charge/discharge variable is linear, the objective function is linear with respect to \( charge\_discharge \). Linear functions are both convex and concave. Therefore, the negation of the profit function is convex.

## Constraints

The constraints for the problem are:

1. Charge and discharge limits:
   $$
   -max\_charge\_discharge \leq charge\_discharge[i] \leq max\_charge\_discharge
   $$
   These are linear inequalities, and sets defined by linear inequalities are convex.

2. Battery capacity constraints:
   $$
   0 \leq state\_of\_charge[i] \leq battery\_capacity
   $$
   Again, these constraints are linear inequalities, which define convex sets.

3. Battery state of charge evolution:
   $$
   state\_of\_charge[i] = state\_of\_charge[i-1] + charge\_discharge[i-1]
   $$
   This constraint is a linear equality, and the set of solutions to a linear equality is a convex set.

## Conclusion

Given that the objective function is convex and the constraints form a convex set, the battery optimization problem is a convex optimization problem. In convex optimization, any local minimum is a global minimum, and such problems can be solved efficiently using algorithms like interior-point methods, gradient descent, or simplex methods.

The convexity of the battery optimization problem ensures that we can find the global optimum solution that provides the maximum profit for battery operation.